In [1]:
from surprise import SVD
from surprise import Dataset
from surprise.model_selection import cross_validate

# Load the movielens-100k dataset (download it if needed).
data = Dataset.load_builtin('ml-100k')

# Use the famous SVD algorithm.
algo = SVD()

# Run 5-fold cross-validation and print results.
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Dataset ml-100k could not be found. Do you want to download it? [Y/n] Y
Trying to download dataset from http://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to /Users/Stefahrencat/.surprise_data/ml-100k
Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9380  0.9406  0.9336  0.9367  0.9332  0.9364  0.0028  
MAE (testset)     0.7366  0.7428  0.7389  0.7399  0.7337  0.7384  0.0031  
Fit time          4.41    4.63    4.38    4.46    4.48    4.47    0.08    
Test time         0.20    0.16    0.12    0.13    0.15    0.15    0.03    


{'test_rmse': array([0.93802442, 0.94062843, 0.9336244 , 0.93673223, 0.93322053]),
 'test_mae': array([0.736568  , 0.74277593, 0.73886194, 0.73994538, 0.73371723]),
 'fit_time': (4.4149250984191895,
  4.6259191036224365,
  4.377200126647949,
  4.462311029434204,
  4.476406097412109),
 'test_time': (0.20047998428344727,
  0.15942883491516113,
  0.12337279319763184,
  0.1252579689025879,
  0.1518871784210205)}